In [1]:
# importing LangChain library to bring together the different components needed for our model

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough , RunnableParallel
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.vectorstores import Qdrant
from qdrant_client import models, QdrantClient
import qdrant_client

In [2]:
# API keys

import os
import dotenv

os.environ["OPENAI_API_KEY"]

In [3]:
# create a qdrant instance

os.environ['QDRANT_HOST']
os.environ["QDRANT_API_KEY"]

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [4]:
# create a qdrant instance

os.environ['QDRANT_HOST'] 
os.environ["QDRANT_API_KEY"]

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [5]:
# read the file

doc ="personas.txt"
data=""

with open(doc,'r') as f:
    data = f.read()

In [6]:
# split the text into chunks
#create a function to return chunks
def get_chunks(text):
    text_splitter=CharacterTextSplitter(
        separator= "\n",
        chunk_size=1000,
        chunk_overlap=100, # second chunk start  character from 800, overlap is used to stop loosing chunk
        length_function=len
    )

    chunks=text_splitter.split_text(text)
    return chunks

In [7]:
# get the chunks for the data
texts=get_chunks(data)

In [ ]:
# creating a new collection

vectors_config=models.VectorParams(
    # depends on model, we can google dimension. 1536 for openai
    # we are using openai embedding, for that size is 1536
    size=1536,
    distance=models.Distance.COSINE)

client.create_collection(
    collection_name="Personas",
    vectors_config=vectors_config,
)

In [8]:
# create a vector store object using langchain

# if we want to use any other embedding, we need to change size

os.environ["OPENAI_API_KEY"]

embeddings = OpenAIEmbeddings()

vector_store = Qdrant(
    client=client,
    collection_name="Personas",
    embeddings=embeddings,
)


e:\AnacondaFile\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [12]:
# add chunks to vector store
vector_store.add_texts(texts)

['b62fbdf665f34e82ae50e3de98f444d0',
 '4c198a6d6d8c423fba2b2158bc7955ec',
 '1592a18c6c81427497a17f53d5b2efbb',
 'de2daab914454c3782440af0be90d6a9',
 'e03c4072a69e4278908d0d045417fe00',
 'ad75eef7fd824945bdbd3b22a706f885',
 'cb9a45933a724f018100414aaae4e334',
 'a8d41e269d164e20ba5bfa1a3d2e5375',
 '138753a625fd47489eb3ed9ee7263459',
 'e635e89bc81e4a7db62dd2758339f943',
 '0a932e0160c74059a2ba8a1b3db670a1',
 '8ee251c4589b43588c50fe8c24588cc3',
 '6366a3f45cd944d795e636070c6ec5fe',
 'ae2863a6de614b13bdb29abcff1583dc',
 '6c8ec70738bb44bb9a66a522df9cfedd',
 '40655d01d0084d6e971295244319ee49',
 '435b2db36c0c424e82a4bc6171922d5f',
 '0904a9970f7c42d2ae4b69e3af620adb',
 'a72d1bccfb634d0db902e6c861fce003',
 'ec95c7e33d97404fa05aa12aad568e6b',
 '96f3273f7c484913a0732730be12be36',
 '15245376f40c48739b560c547cf649e7',
 'f56f45416d1d4f5c92bf2b31b780c9c9',
 '197c2017ac834fa4bafa234f0985bb5c',
 '47b30cb976fd4548babb12cfcdedde5e',
 'efae3f39c5d84bcb897f0033947f6437',
 '33431665598d4ed8a506ec3e3703310e',
 

In [29]:
# create retriever & let's try passing a question through to a these vectorstore objects


retriever = vector_store.as_retriever()

template1 = """"You are an strategizing AI agent for industrial projects 
you provide insights which are categorized based on the personas provided to you. 
Your insights should reflect proper industry experience of the persona
 Please include the personas employed in the team composition.
     {context}

Question: {question}
"""

In [30]:
template2 = """"You are a AI smart assistant here to offer guidance and advice on a variety of tasks and queries. 
Utilize the available tools to move forward in finding the answer to your question
{context}

Question: {question}"""

In [31]:
template = template1 + template2

In [32]:
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [33]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
# invoke response
chain.invoke("How can I set up a project for developing a industrial applications, including needed skills, team members, and budget considerations?")


In [34]:
# stream response

for chunk in chain.stream(
"How can I set up a project for developing a industrial applications, including needed skills, team members, and budget considerations?"):
    print(chunk, end="", flush=True)

To set up a project for developing industrial applications, you will need to consider the following:

1. Define Project Scope and Objectives:
   - Clearly outline the scope of the project, including the specific industrial applications you aim to develop and the objectives you want to achieve.

2. Identify Needed Skills and Expertise:
   - As Manager Maya, leverage your expertise in coordinating cross-functional teams to identify the necessary skills required for the project. This may include software developers, engineers, UI/UX designers, quality assurance testers, and industrial domain experts.

3. Team Composition:
   - As Manager Maya, assemble a diverse and skilled team that complements each other's strengths. Ensure effective communication and collaboration among team members to ensure project success.

4. Budget Considerations:
   - Utilize your experience in delivering projects within budget to develop a detailed budget plan for the industrial applications project. Consider co

In [28]:
# stream response

for chunk in chain.stream(
"Can you advise with how to increase the number of units sold in a supermarket based off of summer sales and deals?"):
    print(chunk, end="", flush=True)

As an AI smart assistant with insights from personas like Manager Max, a social media manager, and User Uma, a hypothetical end-user, here are some strategies to increase the number of units sold in a supermarket during summer sales and deals:

1. Social Media Campaigns: Manager Max can create and execute social media campaigns that promote the summer sales and deals at the supermarket. Highlighting special discounts, limited-time offers, and promotions can attract more customers to visit the store.

2. Compelling Content: Manager Max can develop engaging content that showcases the products on sale, the benefits of shopping during the summer season, and the value of the deals offered. This content can be shared on social media platforms to generate interest and drive traffic to the supermarket.

3. User Experience: User Uma's perspective can help in ensuring a seamless user experience for customers visiting the supermarket. By focusing on usability, intuitiveness, and value, the superm

In [22]:
# stream response

for chunk in chain.stream(
"What strategies should we consider to improve our financial performance?"):
    print(chunk, end="", flush=True)

To improve financial performance, considering the personas provided in the team composition, the following strategies can be considered:

1. **Advisor Amy (Financial Advisor)**:
   - Utilize Amy's expertise in investment planning, retirement planning, estate planning, and risk management to optimize the portfolio and financial strategies.
   - Leverage Amy's personalized financial advice and guidance to tailor financial plans for different client needs and circumstances.
   - Stay updated on market trends, economic developments, and regulatory changes to make informed decisions that benefit the financial performance.

2. **Strategist Sophia (Business Strategist)**:
   - Tap into Sophia's strategic thinking and business acumen to develop long-term strategies that drive business growth and improve financial performance.
   - Analyze market dynamics, competitive landscape, and business models to identify opportunities for revenue generation and cost optimization.
   - Consider future tren

# Persona Selection

In [13]:
# List available Personas

personas = """

- Cybersecurity Expert
- DevOps Engineer
- Data Privacy Officer
- Physicist
- General Engineer
- Skeptic
- Optimist
- Historian
- Wise Guru
- Customer Service Representative
- Hypothetical User
- User Experience Designer
- Product Manager
- Project Manager
- Ethicist
- Business Strategist
- AI Researcher
- Frontend Developer
- Backend Developer
- Data Scientist
- Technical Writer
- Quality Assurance Specialist
- Threat Modeling Expert
- Organizational Cybernetician
- AI Prompt Engineer
- Ontologist & Semantic Architect
- Archaeologist
- Sociologist
- Cryptocurrency Analyst
- Environmental Scientist
- Human Resources Manager
- Financial Advisor
-Crisis Management Specialist
- Urban Planner
- Diversity, Equity, and Inclusion (DEI) Consultant
- Health Policy Analyst
- Social Media Manager
- Supply Chain Analyst
- Ethical Hacker
- Digital Marketing Specialist
- Occupational Therapist
- Renewable Energy Engineer
- Forensic Psychologist
- Fashion Designer
- Clinical Researcher
- Sports Physiotherapist
- Environmental Engineer
- Patent Attorney



"""

In [14]:
#Persona Selection prompt

system_prompt = """
Take into account the nuances and underlying themes of the following question, giving it careful consideration.

Question: {question}

"Carefully choose seven specialized personas from the provided list who possess the expertise required to 
establish a project aimed at developing industrial applications. Consider the individual depth and breadth 
of their skills, as well as the unique insights they offer. Recognize how their collective contributions may 
lead to innovative breakthroughs in addressing project challenges.".

I know you'll do great!

Available Personas: {personas}\n\n{format_instructions}

"""

In [15]:
# # Defines the response schema we want to receive

response_schemas = [
    ResponseSchema(name="persona1", description="the most relevant persona selected to use to reason through the question"),
    ResponseSchema(name="persona2", description="the second persona selected to use to reason through the question"),
    ResponseSchema(name="persona3", description="the third persona selected to use to reason through the question"),
    ResponseSchema(name="persona4", description="the fourth persona selected to use to reason through the question"),
    ResponseSchema(name="persona5", description="the fifth persona selected to use to reason through the question"),
    ResponseSchema(name="persona6", description="the sixth persona selected to use to reason through the question"),
    ResponseSchema(name="persona7", description="the seventh persona selected to use to reason through the question"),
    ResponseSchema(name="rationale", description="for each persona, carefully explain the most compelling reason for including them")

]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [16]:
# Instructions for how the output should be formatted into our initial prompt
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

In [17]:
model = ChatOpenAI(temperature=0.5)
chain = prompt | model | output_parser

In [ ]:
chain.invoke({"question": "Which experts have been selected for this project?"})

In [49]:
for chunk in chain.stream({
"question": "Which experts have been selected for this project?"}):
    print(chunk, end="", flush=True)

{'persona1': 'Data Scientist', 'persona2': 'Machine Learning Engineer', 'persona3': 'Subject Matter Expert', 'persona4': 'Project Manager', 'persona5': 'Software Developer', 'persona6': 'Data Engineer', 'persona7': 'User Experience Designer', 'rationale': {'Data Scientist': 'To analyze and interpret complex data sets to derive insights and make data-driven decisions.', 'Machine Learning Engineer': 'To design and implement machine learning algorithms to develop predictive models.', 'Subject Matter Expert': 'To provide domain-specific knowledge and insights to guide the project in the right direction.', 'Project Manager': 'To oversee the project, coordinate team members, and ensure timely delivery within budget.', 'Software Developer': 'To develop and maintain software applications, ensuring they meet the project requirements.', 'Data Engineer': 'To design, construct, install, test, and maintain highly scalable data management systems.', 'User Experience Designer': 'To focus on enhancing